In [1]:
# @title Подключение к диску с данными
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title Добавление новых фич (исправленная версия)
import polars as pl

# Пути
DATA_PATH = "/content/drive/MyDrive/ml-vtb-data-fusion-strazh/data/"
# Загружаем lazy
df = pl.scan_parquet(f"{DATA_PATH}train_full.parquet")
df = df.sort(["customer_id", "event_dttm"])
# 1. Базовые time-features
df = df.with_columns([
    pl.col("event_dttm").dt.hour().alias("hour"),
    pl.col("event_dttm").dt.weekday().alias("dow"),
    pl.col("event_dttm").dt.date().alias("date"),
    (pl.col("operaton_amt").log1p()).alias("log_amt"),  # опечатка в названии колонки!
])

# 2. Device risk score (с fill_null для null)
df = df.with_columns([
    pl.col("compromised").fill_null("0").cast(pl.Int32),
    pl.col("developer_tools").fill_null("0").cast(pl.Int32),
    pl.col("web_rdp_connection").fill_null("0").cast(pl.Int32),
    pl.col("phone_voip_call_state").fill_null("0").cast(pl.Int32)
])

df = df.with_columns(
    (pl.col("compromised") +
     pl.col("developer_tools") +
     pl.col("web_rdp_connection") +
     pl.col("phone_voip_call_state")).alias("device_risk")
)
# Если хочешь заполнить null в device_risk: .fill_null(0)


# 3. Rolling features по customer_id (отдельно agg + join)
windows = ["1h", "24h", "7d", "30d"]

for w in windows:
    # Сортируем (для rolling) так как перед каждой такой операцией нам нужна строгая сортировка
    df = df.sort(["customer_id", "event_dttm"])
    agg = df.rolling(
        index_column="event_dttm",    # временная колонка
        period=w,                      # размер окна
        group_by="customer_id",        # группировка
        closed="left",                 # не включаем текущую строку (только прошлое)
        #min_periods=1                  # минимум 1 строка для аггрегации
    ).agg([
        pl.col("event_id").count().alias(f"cnt_{w}"),
        pl.col("operaton_amt").sum().alias(f"sum_amt_{w}"),
        pl.col("operaton_amt").mean().alias(f"mean_amt_{w}"),
        pl.col("mcc_code").n_unique().alias(f"uniq_mcc_{w}"),
    ])

    # Join обратно к df
    df = df.join(agg, on=["customer_id", "event_dttm"], how="left")

# 4. Target encoding для категорий (без leak — только прошлое)
cat_cols = ["mcc_code", "event_type_nm", "channel_indicator_type"]
for col in cat_cols:
    df = df.sort(["customer_id", "event_dttm"])
    agg_te = df.rolling(
        index_column="event_dttm",    # временная колонка
        period="30d",                 # размер окна
        group_by=[col, "customer_id"],# группировка
        closed="left",
        #min_periods=1
    ).agg(
        pl.col("target").mean().alias(f"te_{col}_30d")
    )

    # Join обратно (включая col в on, т.к. group_by по ней)
    df = df.join(agg_te, on=[col, "customer_id", "event_dttm"], how="left")

df = df.sort(["customer_id", "event_dttm"])
# Сохраняем (collect в streaming для памяти)
df.collect(engine="streaming").write_parquet(f"{DATA_PATH}train_features.parquet", compression="zstd")